## Homework 3: Mining Data Streams
You are to study and implement a streaming graph processing algorithm described in one of the above papers of your choice.   
In order to accomplished your task, you are to perform the following two steps  

First, implement the reservoir sampling or the Flajolet-Martin algorithm used in the graph algorithm presented in the paper you have selected;  
Second, implement the streaming graph algorithm presented in the paper that make use of the algorithm implemented in the first step.   
#### Group 28
Junjie Shan & Yuxin Meng

In [1]:
import random

### Algorithm
we chose to implement the second paper(both base and impr versions):  
L. De Stefani, A. Epasto, M. Riondato, and E. Upfal, **TRIÈST: Counting Local and Global Triangles in Fully-Dynamic Streams with Fixed Memory Size**, KDD'16.  
link: http://www.kdd.org/kdd2016/papers/files/rfp0465-de-stefaniA.pdf

We implemented two algorithms TRIST_BASE and TRIST_IMPR, which can only deal with insertion-only streams, to estimate the global and local triangles of an ongoing stream of graph (with only insertions).

In [2]:
class TRIEST_BASE:
    def __init__(self, M = 100):
        self.M = M
        self.S = set()
        self.global_counter = 0
        self.local_counter = {}
        self.t = 0

    # Using the reservoir sampling process, and each edge item in the sample has equal probability.
    def sample_edge(self, Edge, t):
        # when t <= M, the edge is deterministically inserted in S
        if t <= self.M:
            return True
        # with probability M/t to be true, it will choose an existing edge in S uniformly at random
        if random.random() <= (self.M/t):
            random_edge = random.sample(self.S, 1)[0]
            # remove it and insert the current edge on time t into S 
            self.S.remove(random_edge)
            self.update_counters('-', Edge)
            return True
        return False

    def update_counters(self, operation, edge):
        # edge E = (u,v)
        u = edge[0]
        v = edge[1]
        neighborhood_of_u = set()
        neighborhood_of_v = set()
        # construct neighborhood of u
        for one_edge in self.S:
            if u == one_edge[0]:
                neighborhood_of_u.add(one_edge[1])
            if u == one_edge[1]:
                neighborhood_of_u.add(one_edge[0])
            # construct neighborhood of v
            if v == one_edge[0]:
                neighborhood_of_v.add(one_edge[1])
            if v == one_edge[1]:
                neighborhood_of_v.add(one_edge[0])
        # shared neighborhood of u and v
        shared_neighborhood = set.intersection(neighborhood_of_u, neighborhood_of_v)
        # update counters
        for c in shared_neighborhood:
            if operation == '+':
                self.global_counter += 1
                self.local_counter[c] = self.local_counter.get(c, 0) + 1
                self.local_counter[u] = self.local_counter.get(u, 0) + 1
                self.local_counter[v] = self.local_counter.get(v, 0) + 1

            if operation == '-':
                self.global_counter -= 1
                self.local_counter[c] = self.local_counter.get(c, 0) - 1
                if self.local_counter[c] <= 0:
                    del self.local_counter[c]
                self.local_counter[u] = self.local_counter.get(u, 0) - 1
                if self.local_counter[u] <= 0:
                    del self.local_counter[u]
                self.local_counter[v] = self.local_counter.get(v, 0) - 1
                if self.local_counter[v] <= 0:
                    del self.local_counter[v]

    def run_triest_base(self, streams):

        for element in streams:
            self.t += 1
            if self.sample_edge(element, self.t):
                self.S.add(element)
                self.update_counters('+', element)

        eps = (self.t * (self.t - 1) * (self.t - 2)) / (self.M * (self.M - 1) * (self.M - 2))

        eps = max(1, eps)
        print('Epsilon is ', eps)
        # estimation for the global triangle count
        est_gc = eps * self.global_counter
        return est_gc

### Data pre-processing
The dataset we used is the High Energy Physics - Theory collaboration network. (https://snap.stanford.edu/data/ca-HepTh.html)    
|dataset statistics||
|  ----  | ----  |
Nodes|	9877
Edges|	25998
Nodes in largest WCC|	8638 (0.875)
Edges in largest WCC|	24827 (0.955)
Nodes in largest SCC|	8638 (0.875)
Edges in largest SCC|	24827 (0.955)
Average clustering coefficient|	0.4714
Number of triangles|	28339
Fraction of closed triangles|	0.1168
Diameter (longest shortest path)|	17
90-percentile effective diameter|	7.4

Although the dataset's information shows that it contains undirected edges, however, in actual file, there are more than 25998 edges.  
So we need to pre-process the dataset, to make the edge, for example, Edge(562, 9890) be the same as Edge(9890, 562)

In [3]:
def undirected_edge(u,v):
    if u < v:
        return (u,v)
    else:
        return (v,u)

# import dataset
streams = set()

with open("CA-HepTh.txt") as f:
    for line in f:
        if line[0] == '#':
            continue
        edge = line.split()
        if edge[0] != edge[1]:
            streams.add(undirected_edge(edge[0], edge[1]))
        # if size_stream == 10000:
        #     break

print('The amount of edges of Data Stream contains', len(streams))


The amount of edges of Data Stream contains 25973


In [4]:
# set M >=6 & M < length of the dataset
# Estimate the global triangle count
triest_base = TRIEST_BASE(3000)
print('the value of M is', triest_base.M)
glo_tri_counter = triest_base.run_triest_base(streams)
print('Estimation for the global triangle count is', glo_tri_counter)

# Get the true amount of global triangles
triest_base = TRIEST_BASE(len(streams))
print('the value of M is', len(streams))
true_glo_tri_counter = triest_base.run_triest_base(streams)
print('Actual global triangle count is', true_glo_tri_counter)

# Compare the estimation & Actual count
error_count = abs(true_glo_tri_counter - glo_tri_counter)
print('Error triangles are', error_count)
error_rate = error_count/ true_glo_tri_counter
print('Error rate is', error_rate)



the value of M is 3000
Epsilon is  649.5114821120048
Estimation for the global triangle count is 28578.505212928212
the value of M is 25973
Epsilon is  1
Actual global triangle count is 28339
Error triangles are 239.50521292821213
Error rate is 0.008451434875197153
